# Large Language Models as Research Agents: Part 3 - Model Context Protocol

*NASA Cosmic Origins AI/ML STIG Tutorial Series*

*Part 3 of 3: MCP for Standardized LLM Tool Integration*

**Learning Objectives:**
- Understand the Model Context Protocol (MCP) for packaging tools as standalone servers
- Build astronomical MCP servers that work with Claude Desktop
- Leverage subscription-based access for cost-free interactive exploration
- Create modular, reusable tool servers for your research workflow
- Expose both calculation tools and data resources through MCP

---

**Attribution:**  
This material is adapted from [*Coding Essentials for Astronomers*](https://tingyuansen.github.io/coding_essential_for_astronomers/) by Yuan-Sen Ting.

**Citation:**  
Ting, Y.-S. (2025). *Coding Essentials for Astronomers*. Zenodo. [DOI: 10.5281/zenodo.17850426](https://doi.org/10.5281/zenodo.17850426)

## Introduction

Cast your mind back to Part 2 of this series. You built function tools that let Claude execute calculations—parallax to distance, stellar luminosity, orbital mechanics. You created schemas, handled tool requests, and sent results back through a careful dance of API calls. It worked beautifully for your astronomical calculations.

But there were two significant drawbacks to that approach.

First, every API call costs money. Each time you ran your notebook, you paid for tokens—for the prompt, for Claude's reasoning, for the response. When you're exploring and experimenting, asking "what if I change this parameter?" over and over, those costs add up. You might find yourself hesitating before running a cell, mentally calculating whether this exploration is worth another few cents. That hesitation is antithetical to good science.

Second, your tools were trapped inside your notebook. The parallax calculator you built couldn't be used anywhere else without copying the code, the schema, the tool-handling logic—everything. Want to use those same calculations in a different project? Copy and paste. Want to share them with a colleague? Send them the whole notebook and hope they can figure it out. Your beautiful astronomical tools became prisoners of the notebook where you defined them.

Today, we solve both problems with the Model Context Protocol—MCP. MCP lets you package your astronomical tools into standalone servers that can be used by any compatible application. Most importantly for your day-to-day work, MCP lets you connect those tools to Claude Desktop, a chat application that uses your **subscription** rather than API credits. You can ask questions, run calculations, and explore freely without watching your wallet.

### The Cost Advantage: Subscription vs API

This point deserves emphasis because it fundamentally changes how you can use LLMs in your research workflow.

When you use the API directly (as covered in Parts 1 and 2), you pay per token. Every character in your prompt costs money. Every character in Claude's response costs money. If Claude reasons through a problem step by step, you pay for each step. This model makes sense for production systems where you're processing thousands of requests automatically, but it creates friction for interactive exploration.

Claude Desktop works differently. With a Pro subscription (currently $20/month), you get generous usage limits for conversational interactions. When you connect your MCP tools to Claude Desktop, those tool calls happen within your subscription—no additional API charges. You can ask "What's the equivalent width?" then immediately follow up with "What if the line were deeper?" then "How does that compare to thermal broadening?" then "Let me try a different temperature"—all without cost anxiety.

This isn't just about saving money (though that's nice). It's about removing barriers to exploration. Science advances through iteration, through "what if" questions, through following unexpected threads. When each question costs money, you unconsciously filter your curiosity. When questions are essentially free, you explore more freely.

| Approach | Cost Model | Best For |
|----------|-----------|----------|
| API (Parts 1-2) | Pay per token | Automated pipelines, batch processing |
| Claude Desktop + MCP | Subscription (flat rate) | Interactive exploration, learning, experimentation |

### MCP vs API: Conversational vs Programmatic

You might wonder: why learn MCP when the API approach from Parts 1-2 already works?

The answer is that they serve fundamentally different purposes, and understanding when to use each will make you more effective.

**The API approach gives you programmatic control.** You write Python code that sends requests and processes responses. This is powerful for automated pipelines—processing hundreds of spectra, batch analyzing observation logs, or building production systems. But you pay per token, and you must manage conversations yourself.

**MCP creates a conversational interface where Claude orchestrates your tools.** Users describe what they want in natural language, and Claude figures out which tools to call and how to combine them. This is excellent for exploration, for questions you didn't anticipate, for combining capabilities in unexpected ways.

Here's a concrete example. Suppose you've built tools for spectroscopic analysis. With the API approach, you'd write code to call each function, process results, and handle the conversation loop. Every exploration costs tokens.

With MCP connected to Claude Desktop, you simply ask:

- "What's the equivalent width for a line with depth 0.4 and FWHM 0.12 Angstroms?"
- "Is that line on the linear part of the curve of growth?"
- "What if the line were twice as deep?"
- "Compare thermal broadening at 5000K vs 6000K for iron lines at 6000 Angstroms"
- "Which of those scenarios would give me better abundance precision?"

Claude orchestrates your tools to answer each question, combining them as needed, explaining the results in context. All within your subscription—no per-token costs.

**The practical guidance:** Use the API (as covered in Parts 1-2) when you need programmatic control for automated pipelines. Use MCP + Claude Desktop when you want flexible, conversational exploration using your subscription. These aren't mutually exclusive—you might use MCP for exploration and prototyping, then switch to the API when you need to process data at scale.

### What You'll Build Today

By the end of this lecture, you'll have:

1. A working MCP server with spectroscopic analysis tools (equivalent width, Doppler shifts, line broadening, curve of growth)
2. Astronomical data resources that expose line lists and reference data to Claude
3. Claude Desktop integration where you can use your tools through natural conversation
4. Understanding of when to use MCP vs the API approach from Parts 1-2

Let's begin.

## Understanding How MCP Works

Before writing code, let's understand what MCP actually does. Don't worry if some concepts seem abstract initially—they'll become concrete once we start building.

### The Client-Server Model

MCP uses a "client-server" architecture. If you've ever used a web browser, you've already experienced this pattern: your browser (the client) requests web pages from websites (the servers). The browser doesn't contain all the websites in the world—it just knows how to ask for them and display what comes back.

MCP works similarly. **MCP Clients** are applications that want to use tools. The main example we'll use is Claude Desktop—a standalone chat application from Anthropic that you install on your computer. When you chat with Claude Desktop and it needs to perform a calculation, it can ask an MCP server to do that calculation. Other clients exist too: IDE extensions, custom applications, even other chat interfaces. Any application that implements the MCP protocol can be a client.

**MCP Servers** are programs that provide tools and data. This is what you'll build: a Python program containing your astronomical functions. The server waits for requests, executes the appropriate function when asked, and sends back results. A server can provide multiple tools, and it can also provide data resources (like line lists or reference tables).

The key insight is **separation**. Your astronomical functions live in the server. The chat interface lives in the client. They communicate through a defined protocol (MCP), which means they can be developed, updated, and used independently. You could update your server with new tools without changing anything about Claude Desktop. You could use the same server with a different client application. This modularity is powerful.

### Why Separation Matters

In Part 2, your tools were embedded directly in your notebook. The function, the schema, and the conversation-handling logic were all intertwined. This works fine for a single notebook, but consider what happens when you want to use the same calculations while chatting in Claude Desktop, share your tools with a colleague who uses different software, add your tools to a computing cluster, or update a calculation without breaking existing code that uses it.

With embedded tools, each scenario requires copying and adapting code. With MCP, you write your tools once as a server, and any compatible client can use them. Update the server, and all clients automatically get the improvement.

### How Communication Actually Works

When Claude Desktop wants to use one of your tools, here's the sequence of events:

1. **You ask a question.** "What's the Doppler shift of H-alpha if a star is receding at 100 km/s?"

2. **Claude Desktop connects to your server.** When Claude Desktop starts, it launches your MCP server as a subprocess and establishes a communication channel.

3. **Claude Desktop asks what tools are available.** The server responds with descriptions of all available tools—similar to the schemas you wrote in Part 2, but generated automatically from your Python code.

4. **Claude recognizes a calculation is needed.** Based on your question, Claude decides it should use a tool rather than just answering from memory.

5. **Claude picks the right tool.** Looking at the available tools, Claude identifies `doppler_shift` as the appropriate one.

6. **Claude Desktop sends the request.** It tells your server: "Please run `doppler_shift` with `rest_wavelength_angstroms=6562.8` and `velocity_kms=100`."

7. **Your server executes the function.** Your Python code runs, computing the result.

8. **Your server returns the result.** The calculated values go back to Claude Desktop.

9. **Claude formulates a natural language response.** Claude takes the numerical result and writes an explanation for you: "At 100 km/s recession velocity, H-alpha at 6562.8 Å would be observed at 6564.99 Å—a redshift of about 2.19 Å."

All of this happens automatically once you've set things up. You just chat naturally.

### Tools vs Resources

MCP servers can provide two kinds of capabilities. **Tools** are functions that perform calculations or actions—they take inputs and return outputs. Your `doppler_shift` function is a tool: give it a wavelength and velocity, get back an observed wavelength. **Resources** are data that clients can read, identified by URIs (like web addresses). A line list for iron absorption lines could be a resource—Claude can read it to know what lines are available, then use your tools to calculate properties of specific lines.

The distinction matters for astronomical work. Your research often involves both calculations (tools) and reference data (resources). MCP lets you expose both through the same server.

### A Note on Terminology: Transports

You might encounter the word "transport" in MCP documentation. This refers to how the client and server send messages to each other. The most common transport is "stdio" (standard input/output)—Claude Desktop launches your server as a subprocess and communicates through text pipes. You don't need to understand the details; the MCP library handles everything. Just know that when documentation mentions "stdio transport," it means "the normal way Claude Desktop talks to local servers."

## Setting Up Your Environment

Let's install the MCP library. This handles all the communication protocol details so you can focus on writing astronomical functions.

In [ ]:
# Install the MCP SDK (Software Development Kit)
# This provides the tools we need to create MCP servers
%pip install mcp

In [ ]:
# Verify the installation
from importlib.metadata import version

try:
    mcp_version = version('mcp')
    print(f"MCP SDK version: {mcp_version}")
    
    # Verify we can import the main class
    from mcp.server.fastmcp import FastMCP
    print("✓ MCP SDK is ready to use")
    
except ImportError as e:
    print(f"✗ Installation problem: {e}")
    print("\nTroubleshooting steps:")
    print("1. Try: pip install mcp")
    print("2. If that fails, create a fresh conda environment:")
    print("   conda create -n mcp-env python=3.11")
    print("   conda activate mcp-env")
    print("   pip install mcp")

### Understanding the Key Components

The MCP library provides several building blocks. For most astronomical applications, you'll use three:

**`FastMCP`**: A class that creates an MCP server. Think of it as the container that holds your tools and handles all communication. You create one FastMCP instance, register your functions with it, and it manages everything else.

**`@mcp.tool()` decorator**: A decorator that marks a Python function as an MCP tool. Decorators are the `@something` syntax you put above function definitions—they modify how the function behaves. In this case, `@mcp.tool()` tells MCP "this function should be available as a tool that Claude can call."

**`@mcp.resource()` decorator**: Similar, but for data resources. You mark a function that returns data, and MCP makes that data available for clients to read.

**The key insight: No manual schemas required.** Remember in Part 2, you had to write JSON schemas like this for every function?

```python
# Part 2 approach - manual schema for EACH function
parallax_tool = {
    "name": "calculate_distance",
    "description": "Calculate distance from parallax",
    "input_schema": {
        "type": "object",
        "properties": {
            "parallax_arcsec": {
                "type": "number",
                "description": "Parallax angle in arcseconds"
            }
        },
        "required": ["parallax_arcsec"]
    }
}
```

With MCP, you write none of that. The `@mcp.tool()` decorator automatically reads your function's **name**, **type hints**, and **docstring** to generate the schema. Your docstring becomes the tool description. Your parameter type hints become the input schema. This means writing good docstrings isn't just good practice—it's how you tell Claude what your tool does.

Let's see this in action.

## Your First MCP Server

We'll build a simple server providing equivalent width estimation—a common spectroscopic calculation, packaged as an MCP tool.

### Creating the Server File

MCP servers are standalone Python files that run independently of your notebook. We need to save our server code to a file. The `%%writefile` magic command does this—it saves the cell contents to a file instead of executing them. Let's create a simple server with one tool:

In [ ]:
%%writefile ew_server.py
#!/usr/bin/env python3
"""
MCP Server for Equivalent Width Estimation

This server provides spectroscopic calculation tools accessible
through Claude Desktop or other MCP-compatible applications.

The tools here implement the Gaussian approximation for equivalent
width, which is useful for quick estimates and planning observations.
"""

import numpy as np
from mcp.server.fastmcp import FastMCP

# Create the MCP server
# The name appears in Claude Desktop to identify this server
mcp = FastMCP("Spectral Line Analyzer")


# The @mcp.tool() decorator registers this function as a tool
# MCP automatically reads the function signature and docstring
# to create a description that Claude can understand
@mcp.tool()
def estimate_ew_from_depth(
    line_depth: float,
    line_fwhm_angstroms: float
) -> dict:
    """
    Estimate equivalent width from line depth and width.
    
    This uses the Gaussian approximation: EW ≈ 1.064 × depth × FWHM.
    It's a quick estimate useful for planning observations and sanity
    checks. For publication-quality measurements, use proper profile
    fitting for publication-quality measurements.
    
    Args:
        line_depth: Fractional depth of the absorption line, from 0 to 1.
                   A depth of 0.3 means the line absorbs 30% of the continuum.
        line_fwhm_angstroms: Full Width at Half Maximum in Angstroms.
                            This measures how broad the line appears.
        
    Returns:
        Dictionary containing:
        - ew_mA: Equivalent width in milli-Angstroms
        - ew_angstroms: Equivalent width in Angstroms
        - caveat: Assessment of whether this estimate is reliable
    """
    # Validate inputs - astronomical quantities have physical constraints
    if line_depth <= 0 or line_depth > 1:
        return {"error": "Line depth must be between 0 and 1"}
    if line_fwhm_angstroms <= 0:
        return {"error": "FWHM must be positive"}
    
    # Gaussian approximation for equivalent width
    # The factor 1.064 comes from integrating a Gaussian profile
    ew_angstroms = 1.064 * line_depth * line_fwhm_angstroms
    ew_mA = ew_angstroms * 1000  # Convert to milli-Angstroms
    
    # Provide context about reliability
    if line_depth > 0.7:
        caveat = "Line may be saturated—Voigt profile fitting recommended"
    elif line_depth < 0.1:
        caveat = "Weak line—measurement uncertainty may be significant"
    else:
        caveat = "Gaussian approximation should be reasonable for this line strength"
    
    return {
        "ew_mA": round(ew_mA, 2),
        "ew_angstroms": round(ew_angstroms, 4),
        "caveat": caveat
    }


# This block runs when the file is executed directly
# mcp.run() starts the server and waits for connections
if __name__ == "__main__":
    mcp.run()

### Understanding the Code Structure

Let's walk through the key parts of what we just wrote.

The **imports and server creation** at the top (`from mcp.server.fastmcp import FastMCP` and `mcp = FastMCP("Spectral Line Analyzer")`) create an MCP server object. The name you provide ("Spectral Line Analyzer") appears in Claude Desktop to help you identify which server provides which tools.

The **tool decorator** (`@mcp.tool()`) transforms your function into an MCP tool. MCP reads your function's name, parameters, type hints, and docstring to automatically generate the tool description. This is similar to the schemas you wrote manually in Part 2, but automated.

**Type hints are essential**. The `: float` annotations (like `line_depth: float`) tell MCP what kind of data each parameter expects. Without these, MCP can't properly describe the tool to Claude. Always include type hints for MCP tools.

**The docstring is crucial**. Your docstring is what Claude reads to understand when and how to use the tool. Write it as you would for a knowledgeable colleague—clear, with physical context, including units. Claude uses this description to decide whether your tool is appropriate for a given question.

The **`if __name__ == "__main__":` block** at the bottom is a Python pattern meaning "only run this code if the file is executed directly." When you import from this file (like we'll do for testing), this block doesn't run. But when Claude Desktop launches the server, it executes the file directly, which triggers `mcp.run()` to start the server.

### Testing the Function

Before connecting to Claude Desktop, let's verify our function works correctly. We can import directly from the file we created:

In [ ]:
# Import our function from the file we created
from ew_server import estimate_ew_from_depth

# Test with typical values
print("Testing estimate_ew_from_depth function:\n")

# Test case 1: A moderate absorption line
result = estimate_ew_from_depth(line_depth=0.3, line_fwhm_angstroms=0.15)
print(f"Input: depth=0.3, FWHM=0.15 Å")
print(f"Result: {result}")

# Verify the math manually: EW = 1.064 × 0.3 × 0.15 × 1000 mÅ
expected = 1.064 * 0.3 * 0.15 * 1000
print(f"Expected EW: {expected:.2f} mÅ")
print(f"Match: {'✓' if abs(result['ew_mA'] - expected) < 0.01 else '✗'}")

In [ ]:
# Test edge cases to verify our validation works
print("Testing edge cases:\n")

# Strong (potentially saturated) line
result_strong = estimate_ew_from_depth(0.8, 0.2)
print(f"Strong line (depth=0.8):")
print(f"  EW: {result_strong['ew_mA']} mÅ")
print(f"  Caveat: {result_strong['caveat']}")

print()

# Weak line  
result_weak = estimate_ew_from_depth(0.05, 0.1)
print(f"Weak line (depth=0.05):")
print(f"  EW: {result_weak['ew_mA']} mÅ")
print(f"  Caveat: {result_weak['caveat']}")

print()

# Invalid input (should return error)
result_invalid = estimate_ew_from_depth(-0.1, 0.1)
print(f"Invalid input (depth=-0.1):")
print(f"  Result: {result_invalid}")

### What Changed from Part 2

Let's appreciate how dramatically simpler this is compared to Part 2's approach.

**In Part 2**, creating a usable tool required seven separate pieces:

1. Writing the function itself
2. Writing a separate JSON schema describing the function's parameters
3. Adding code to detect when Claude wanted to use a tool (checking `stop_reason`)
4. Adding code to extract the tool request and parse the arguments
5. Adding code to execute the function with those arguments
6. Adding code to send results back to Claude in the right format
7. Managing the conversation loop manually

**With MCP**, you need just one thing: write your function with the `@mcp.tool()` decorator and a good docstring. That's it.

Here's a concrete comparison. To create an equivalent width tool:

**Part 2 (manual schema + handling code):**
```python
# 1. The function
def estimate_ew(depth, fwhm):
    return 1.064 * depth * fwhm * 1000

# 2. The schema (you had to write this separately!)
ew_tool = {
    "name": "estimate_ew",
    "description": "Estimate equivalent width from depth and FWHM",
    "input_schema": {
        "type": "object",
        "properties": {
            "depth": {"type": "number", "description": "Line depth 0-1"},
            "fwhm": {"type": "number", "description": "FWHM in Angstroms"}
        },
        "required": ["depth", "fwhm"]
    }
}

# 3-7. Plus all the tool handling code...
```

**MCP (everything in one place):**
```python
@mcp.tool()
def estimate_ew(depth: float, fwhm: float) -> dict:
    """Estimate equivalent width from depth and FWHM.
    
    Args:
        depth: Line depth (0-1)
        fwhm: FWHM in Angstroms
    """
    return {"ew_mA": 1.064 * depth * fwhm * 1000}
```

The decorator reads your type hints (`float`) and docstring to generate the schema automatically. The MCP library handles all the communication. Claude Desktop handles the conversation loop. Your job is just to write good functions with clear docstrings—which you should be doing anyway!

## Connecting to Claude Desktop

Now we'll make your server available to Claude Desktop. This is where the subscription-based, interactive exploration becomes possible.

### What is Claude Desktop?

Throughout this series, you've interacted with Claude in two ways: the **web interface** at claude.ai—you type in a chat box and get responses, simple and convenient, but you can't connect it to your own tools; and the **API** (Parts 1-2)—you write Python code that sends requests and processes responses programmatically, powerful and automatable, but you pay per token and must manage conversations yourself.

**Claude Desktop** is a third option. It's a standalone application you install on your computer. It looks and feels like the web interface—a chat window where you type questions and get responses—but with crucial differences: it can connect to MCP servers running on your machine, it uses your subscription rather than API credits, and Claude can automatically use your tools when answering questions.

This combination makes Claude Desktop ideal for research exploration: unlimited conversational interactions with your own astronomical tools, at no incremental cost.

### Installing and Launching Claude Desktop

Download Claude Desktop from: **https://claude.ai/download**

It's available for macOS, Windows, and Linux. Install it like any other application. When you first open it, sign in with your Anthropic account (the same one you use at claude.ai).

Let's write some helper functions to detect your operating system and manage Claude Desktop. The following code uses several Python standard library modules that are useful for cross-platform programming:

- **`platform.system()`**: Returns the operating system name—"Darwin" for macOS, "Windows" for Windows, "Linux" for Linux. This lets us write code that behaves differently on each OS.

- **`os.path.expanduser("~")`**: Expands the `~` symbol to your home directory path. On macOS this might be `/Users/yourname`, on Linux `/home/yourname`, on Windows `C:\Users\yourname`.

- **`os.environ.get("VARNAME", default)`**: Reads environment variables. Windows uses variables like `APPDATA` and `LOCALAPPDATA` to store standard paths. The second argument provides a fallback if the variable isn't set.

- **`os.path.join()`**: Joins path components using the correct separator for your OS (`/` on Mac/Linux, `\` on Windows).

- **`sys.executable`**: The path to the Python interpreter currently running your code. This is important because Claude Desktop needs to know exactly which Python to use.

In [ ]:
import os
import sys
import json
import platform
import subprocess

def get_system_info():
    """Detect operating system and relevant paths."""
    system = platform.system()
    
    if system == "Darwin":  # macOS
        config_path = os.path.expanduser(
            "~/Library/Application Support/Claude/claude_desktop_config.json"
        )
        app_path = "/Applications/Claude.app"
        launch_command = ["open", "-a", "Claude"]
        
    elif system == "Windows":
        config_path = os.path.join(
            os.environ.get("APPDATA", ""),
            "Claude",
            "claude_desktop_config.json"
        )
        app_path = os.path.join(
            os.environ.get("LOCALAPPDATA", ""),
            "Programs",
            "Claude",
            "Claude.exe"
        )
        launch_command = [app_path]
        
    elif system == "Linux":
        config_path = os.path.expanduser(
            "~/.config/Claude/claude_desktop_config.json"
        )
        app_path = "/usr/bin/claude"  # May vary by installation
        launch_command = ["claude"]
        
    else:
        raise OSError(f"Unsupported operating system: {system}")
    
    return {
        "system": system,
        "config_path": config_path,
        "app_path": app_path,
        "launch_command": launch_command,
        "python_path": sys.executable
    }

# Display your system information
info = get_system_info()
print(f"Operating System: {info['system']}")
print(f"Python interpreter: {info['python_path']}")
print(f"Claude Desktop config: {info['config_path']}")
print(f"Claude Desktop app: {info['app_path']}")

### Configuring Claude Desktop Automatically

Claude Desktop needs to know where your MCP servers are. This is configured through a JSON file whose location varies by operating system. Rather than editing this file manually (which is error-prone and tedious), let's write Python code to do it automatically.

The function below handles several important tasks:

- **Creates the config directory** if it doesn't exist (`os.makedirs` with `exist_ok=True`)
- **Loads existing config** if present, so we don't overwrite other servers you might have configured
- **Uses `json.load()` and `json.dump()`** to read and write JSON files (Python dictionaries map directly to JSON objects)
- **Uses absolute paths** via `os.path.abspath()` so the config works regardless of your current directory

Now let's configure our simple EW server:

In [ ]:
def setup_mcp_server(server_name, server_file, info=None):
    """
    Configure Claude Desktop to use an MCP server.
    
    This function:
    1. Finds (or creates) the Claude Desktop config file
    2. Adds your server to the configuration
    3. Preserves any existing server configurations
    
    Args:
        server_name: A name to identify your server (e.g., "spectral-tools")
        server_file: Path to your MCP server Python file
        info: System info dict (auto-detected if not provided)
    
    Returns:
        Path to the config file
    """
    if info is None:
        info = get_system_info()
    
    # Get full absolute paths
    server_file = os.path.abspath(server_file)
    config_path = info['config_path']
    python_path = info['python_path']
    
    # Ensure the config directory exists
    config_dir = os.path.dirname(config_path)
    os.makedirs(config_dir, exist_ok=True)
    
    # Load existing config or create empty one
    if os.path.exists(config_path):
        with open(config_path, 'r') as f:
            try:
                config = json.load(f)
                print(f"Found existing config file")
            except json.JSONDecodeError:
                print(f"Warning: Existing config was invalid, creating new one")
                config = {}
    else:
        config = {}
        print(f"Creating new config file")
    
    # Ensure mcpServers section exists
    if "mcpServers" not in config:
        config["mcpServers"] = {}
    
    # Add or update this server
    config["mcpServers"][server_name] = {
        "command": python_path,
        "args": [server_file]
    }
    
    # Write the config
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"\n✓ Configuration saved to:\n  {config_path}")
    print(f"\nServer '{server_name}' configured:")
    print(f"  Python: {python_path}")
    print(f"  Server: {server_file}")
    
    return config_path

In [ ]:
# Set up the configuration for our server
setup_mcp_server("spectral-tools", "ew_server.py")

### Verifying and Viewing the Configuration

It's always good practice to verify that configuration files were written correctly before relying on them. Let's examine what was written to the Claude Desktop config file. This also helps you understand the JSON structure in case you ever need to edit it manually:

In [ ]:
# Read and display the current configuration
info = get_system_info()

if os.path.exists(info['config_path']):
    with open(info['config_path'], 'r') as f:
        config = json.load(f)
    
    print("Current Claude Desktop configuration:\n")
    print(json.dumps(config, indent=2))
else:
    print("Configuration file not found. Run the setup cell above first.")

### Testing That the Server Runs

Before connecting to Claude Desktop, it's essential to verify that your server file has no syntax errors or import problems. If the server fails to start, Claude Desktop will silently ignore it—you'll just see that no tools are available. Testing the file directly catches these issues early:

In [ ]:
import importlib.util

def test_server_file(server_path):
    """Test that a server file can be loaded without errors."""
    server_path = os.path.abspath(server_path)
    
    try:
        spec = importlib.util.spec_from_file_location("test_server", server_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        print(f"✓ Server file loads successfully")
        print(f"  Path: {server_path}")
        
        # Check if it has the expected MCP components
        if hasattr(module, 'mcp'):
            print(f"✓ MCP server object found")
            
        return True
        
    except Exception as e:
        print(f"✗ Error loading server: {e}")
        return False

test_server_file("ew_server.py")

### Launching Claude Desktop

Now let's launch Claude Desktop programmatically so you don't forget to restart it after configuration changes. The code uses `subprocess.Popen()` which starts a new process (the Claude Desktop application) without waiting for it to finish—this lets your notebook continue running while Claude Desktop starts up in the background.

On macOS, we use `["open", "-a", "Claude"]` which is the standard way to launch applications. On Windows, we directly run the executable. On Linux, we call the `claude` command.

After launching, you'll need to wait a moment for Claude Desktop to fully start and connect to your server:

In [ ]:
def launch_claude_desktop(info=None):
    """
    Launch Claude Desktop application.
    
    Note: If Claude Desktop is already running, this will bring it to focus.
    If you've changed the config, you need to quit and relaunch for changes
    to take effect.
    """
    if info is None:
        info = get_system_info()
    
    system = info['system']
    
    try:
        if system == "Darwin":  # macOS
            subprocess.Popen(["open", "-a", "Claude"])
            print("✓ Launching Claude Desktop on macOS...")
            
        elif system == "Windows":
            app_path = info['app_path']
            if os.path.exists(app_path):
                subprocess.Popen([app_path], shell=True)
                print("✓ Launching Claude Desktop on Windows...")
            else:
                print(f"✗ Claude Desktop not found at: {app_path}")
                print("  Please launch Claude Desktop manually")
                return
                
        elif system == "Linux":
            subprocess.Popen(["claude"])
            print("✓ Launching Claude Desktop on Linux...")
            
        print("\n📋 Next steps:")
        print("   1. Wait for Claude Desktop to fully start")
        print("   2. Click the slider-like button (bottom-left) to verify your servers are connected")
        print("   3. Try asking questions that use your configured tools!")
        
    except FileNotFoundError:
        print("✗ Could not launch Claude Desktop automatically")
        print("  Please launch it manually from your Applications folder")
    except Exception as e:
        print(f"✗ Error launching Claude Desktop: {e}")

# Launch Claude Desktop
launch_claude_desktop()

### Using Your Tool in Claude Desktop

After Claude Desktop starts, you can verify your MCP servers are connected by clicking the **⊕ button** in the bottom-left of the chat input. This opens a menu showing available integrations—your configured servers should appear here. If you don't see them, try quitting Claude Desktop completely (Cmd+Q on Mac) and relaunching.

Once connected, simply ask Claude questions that need your tools:
- "What's the equivalent width for a line with depth 0.35 and FWHM 0.12 Angstroms?"
- "I measured a weak line with only 5% depth and 0.1 Angstrom width. What's the EW?"
- "What happens to the equivalent width if I double the line depth?"

Claude should automatically recognize these need your calculation tool and use it.

### Troubleshooting

If your server doesn't connect, here are common issues and solutions:

**"spawn python ENOENT" error**: Claude Desktop can't find Python. Our setup function uses the full path to your Python interpreter, which should avoid this. If you still see it, check that the Python path in the config is correct.

**Server not appearing**: Make sure you completely quit Claude Desktop (Cmd+Q on Mac, not just closing the window) and restarted it. The config is only read at startup.

**Import errors in server**: If your server imports packages that aren't installed, it will fail silently. Test the server in your terminal with `python /path/to/ew_server.py`. It should start and wait silently. If you see import errors, install the missing packages.

**Config file not found**: Make sure Claude Desktop has been run at least once before. The configuration directory is created when Claude Desktop first runs.

## Building More Tools: The Modular Approach

Now that you have one working server, you might want to add more tools. Here's an important insight: **Claude Desktop can connect to multiple servers simultaneously**. Rather than putting all your tools in one giant file, you can create focused servers for different purposes:

- `ew_server.py` — equivalent width calculations
- `doppler_server.py` — Doppler shift and velocity calculations
- `broadening_server.py` — line broadening calculations
- `data_server.py` — reference data and line lists

This modular approach has several advantages:
1. **Each file stays small and readable** — easier to understand and debug
2. **You can update one server without touching others** — less risk of breaking things
3. **You can share individual servers** — give a colleague just the Doppler tools
4. **Servers can be developed independently** — work on broadening while Doppler is stable

Let's build these focused servers. First, Doppler calculations:

In [ ]:
%%writefile doppler_server.py
#!/usr/bin/env python3
"""
MCP Server for Doppler Calculations

Provides tools for wavelength shifts and radial velocity measurements.
"""

import numpy as np
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Doppler Tools")
C_KMS = 299792.458  # Speed of light in km/s


@mcp.tool()
def doppler_shift(
    rest_wavelength_angstroms: float,
    velocity_kms: float
) -> dict:
    """
    Calculate observed wavelength given rest wavelength and radial velocity.
    
    Uses the relativistic Doppler formula for accuracy at all velocities.
    Positive velocity means the source is receding (redshift).
    
    Args:
        rest_wavelength_angstroms: Laboratory wavelength in Angstroms
                                  (e.g., H-alpha = 6562.8 Å)
        velocity_kms: Radial velocity in km/s (positive = receding)
        
    Returns:
        Observed wavelength and shift direction
    """
    if rest_wavelength_angstroms <= 0:
        return {"error": "Wavelength must be positive"}
    
    beta = velocity_kms / C_KMS
    if abs(beta) >= 1:
        return {"error": "Velocity cannot exceed speed of light"}
    
    doppler_factor = np.sqrt((1 + beta) / (1 - beta))
    observed = rest_wavelength_angstroms * doppler_factor
    shift = observed - rest_wavelength_angstroms
    
    direction = "redshift" if velocity_kms > 0 else "blueshift" if velocity_kms < 0 else "none"
    
    return {
        "observed_wavelength_angstroms": round(float(observed), 4),
        "shift_angstroms": round(float(shift), 4),
        "direction": direction
    }


@mcp.tool()
def wavelength_to_velocity(
    rest_wavelength_angstroms: float,
    observed_wavelength_angstroms: float
) -> dict:
    """
    Calculate radial velocity from observed wavelength shift.
    
    This is the inverse of doppler_shift—given where a line should be
    and where you observe it, calculate how fast the source is moving.
    
    Args:
        rest_wavelength_angstroms: Laboratory wavelength
        observed_wavelength_angstroms: Measured wavelength in spectrum
        
    Returns:
        Radial velocity in km/s and redshift z
    """
    if rest_wavelength_angstroms <= 0 or observed_wavelength_angstroms <= 0:
        return {"error": "Wavelengths must be positive"}
    
    z = (observed_wavelength_angstroms - rest_wavelength_angstroms) / rest_wavelength_angstroms
    z_factor = (1 + z) ** 2
    beta = (z_factor - 1) / (z_factor + 1)
    velocity = beta * C_KMS
    
    direction = "receding" if velocity > 0 else "approaching" if velocity < 0 else "stationary"
    
    return {
        "velocity_kms": round(float(velocity), 3),
        "redshift_z": round(float(z), 6),
        "direction": direction
    }


if __name__ == "__main__":
    mcp.run()


Next, line broadening calculations. These are separate from Doppler because they address different physics—thermal motion of atoms and stellar rotation rather than bulk motion of the source:

In [ ]:
%%writefile broadening_server.py
#!/usr/bin/env python3
"""
MCP Server for Line Broadening Calculations

Provides tools for thermal and rotational broadening analysis.
"""

import numpy as np
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Broadening Tools")
C_KMS = 299792.458


@mcp.tool()
def thermal_broadening(
    wavelength_angstroms: float,
    temperature_kelvin: float,
    atomic_mass_amu: float
) -> dict:
    """
    Calculate thermal (Doppler) broadening of a spectral line.
    
    Atoms in a hot gas move randomly due to thermal motion, broadening
    spectral lines. Hotter gas = faster atoms = broader lines.
    Heavier atoms move slower at the same temperature = narrower lines.
    
    Args:
        wavelength_angstroms: Line wavelength in Angstroms
        temperature_kelvin: Gas temperature in Kelvin
        atomic_mass_amu: Atomic mass in amu (H=1, Fe=55.845, etc.)
        
    Returns:
        Thermal broadening FWHM in Angstroms and km/s
    """
    if wavelength_angstroms <= 0 or temperature_kelvin <= 0 or atomic_mass_amu <= 0:
        return {"error": "All parameters must be positive"}
    
    k_B = 1.380649e-23  # Boltzmann constant
    amu_kg = 1.66054e-27
    mass_kg = atomic_mass_amu * amu_kg
    
    sigma_v = np.sqrt(2 * k_B * temperature_kelvin / mass_kg)
    fwhm_kms = 2.355 * sigma_v / 1000
    fwhm_angstroms = wavelength_angstroms * fwhm_kms / C_KMS
    
    return {
        "fwhm_angstroms": round(float(fwhm_angstroms), 4),
        "fwhm_kms": round(float(fwhm_kms), 3),
        "sigma_kms": round(float(fwhm_kms / 2.355), 3)
    }


@mcp.tool()
def rotational_broadening(
    wavelength_angstroms: float,
    vsini_kms: float
) -> dict:
    """
    Estimate line broadening from stellar rotation.
    
    One stellar limb moves toward us (blueshift), the other away (redshift),
    broadening spectral lines. We measure 'v sin i'—rotation speed times
    sin(inclination)—since we can't separate these from spectra alone.
    
    Args:
        wavelength_angstroms: Line wavelength in Angstroms
        vsini_kms: Projected rotational velocity in km/s
                  (Sun ≈ 2, typical F star ≈ 20-50, hot stars > 200)
        
    Returns:
        Rotational broadening estimate and rotation classification
    """
    if wavelength_angstroms <= 0:
        return {"error": "Wavelength must be positive"}
    if vsini_kms < 0:
        return {"error": "v sin i must be non-negative"}
    
    delta_lambda = wavelength_angstroms * vsini_kms / C_KMS
    fwhm = 1.8 * delta_lambda
    
    if vsini_kms < 5:
        rot_class = "Slow rotator (Sun-like)"
    elif vsini_kms < 50:
        rot_class = "Moderate rotator"
    elif vsini_kms < 150:
        rot_class = "Fast rotator"
    else:
        rot_class = "Very fast rotator"
    
    return {
        "fwhm_angstroms": round(float(fwhm), 4),
        "delta_lambda_angstroms": round(float(delta_lambda), 4),
        "rotation_class": rot_class
    }


if __name__ == "__main__":
    mcp.run()


Finally, a server for spectral line analysis—equivalent width estimation and curve of growth assessment. These tools help you interpret line measurements and decide which lines are suitable for abundance work:

In [ ]:
%%writefile analysis_server.py
#!/usr/bin/env python3
"""
MCP Server for Spectral Line Analysis

Provides tools for equivalent width estimation and curve of growth analysis.
"""

import numpy as np
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Analysis Tools")


@mcp.tool()
def estimate_ew_from_depth(
    line_depth: float,
    line_fwhm_angstroms: float
) -> dict:
    """
    Estimate equivalent width using the Gaussian approximation.
    
    Args:
        line_depth: Fractional depth (0-1, where 0.3 = 30% absorption)
        line_fwhm_angstroms: Full Width at Half Maximum in Angstroms
        
    Returns:
        Equivalent width in milli-Angstroms with reliability note
    """
    if line_depth <= 0 or line_depth > 1:
        return {"error": "Line depth must be between 0 and 1"}
    if line_fwhm_angstroms <= 0:
        return {"error": "FWHM must be positive"}
    
    ew_angstroms = 1.064 * line_depth * line_fwhm_angstroms
    ew_mA = ew_angstroms * 1000
    
    if line_depth > 0.7:
        caveat = "Line may be saturated—Voigt profile fitting recommended"
    elif line_depth < 0.1:
        caveat = "Weak line—measurement uncertainty may be significant"
    else:
        caveat = "Gaussian approximation should be reasonable"
    
    return {
        "ew_mA": round(float(ew_mA), 2),
        "ew_angstroms": round(float(ew_angstroms), 4),
        "caveat": caveat
    }


@mcp.tool()
def curve_of_growth_regime(
    ew_mA: float,
    wavelength_angstroms: float
) -> dict:
    """
    Determine which part of the curve of growth a spectral line is on.
    
    The curve of growth relates equivalent width to abundance. Weak lines
    are on the linear part (ideal). Strong lines saturate. Very strong
    lines develop damping wings.
    
    Args:
        ew_mA: Equivalent width in milli-Angstroms
        wavelength_angstroms: Line wavelength in Angstroms
        
    Returns:
        Curve of growth regime and recommendation
    """
    if ew_mA <= 0 or wavelength_angstroms <= 0:
        return {"error": "Both values must be positive"}
    
    reduced_ew = (ew_mA / 1000) / wavelength_angstroms
    log_rew = np.log10(reduced_ew)
    
    if log_rew < -5.5:
        regime = "Linear (weak line)"
        desc = "EW proportional to abundance—ideal for analysis"
    elif log_rew < -4.8:
        regime = "Transition"
        desc = "Between linear and saturated—usable but less sensitive"
    elif log_rew < -4.2:
        regime = "Saturated"
        desc = "Line core saturated—poor abundance sensitivity"
    else:
        regime = "Damping wings"
        desc = "Very strong—damping wings dominate"
    
    return {
        "regime": regime,
        "log_reduced_ew": round(float(log_rew), 2),
        "description": desc
    }


if __name__ == "__main__":
    mcp.run()


### Testing the Modular Servers

Let's verify all our focused servers work correctly. Notice how we import from each separate file:

In [ ]:
# Test the modular servers
from doppler_server import doppler_shift, wavelength_to_velocity
from broadening_server import thermal_broadening, rotational_broadening
from analysis_server import estimate_ew_from_depth, curve_of_growth_regime

print("=" * 60)
print("Testing Modular Spectroscopy Servers")
print("=" * 60)

# Test Doppler server
print("\n1. DOPPLER SERVER")
print("-" * 40)
result = doppler_shift(6562.8, 100)
print(f"   H-alpha at +100 km/s: {result['observed_wavelength_angstroms']} Å ({result['direction']})")

result = wavelength_to_velocity(6562.8, 6564.99)
print(f"   6564.99 Å observed: {result['velocity_kms']} km/s ({result['direction']})")

# Test Broadening server
print("\n2. BROADENING SERVER")
print("-" * 40)
result = thermal_broadening(6000, 5500, 55.845)
print(f"   Fe at 5500K: FWHM = {result['fwhm_angstroms']} Å ({result['fwhm_kms']} km/s)")

result = rotational_broadening(6000, 25)
print(f"   v sin i = 25 km/s: FWHM = {result['fwhm_angstroms']} Å ({result['rotation_class']})")

# Test Analysis server
print("\n3. ANALYSIS SERVER")
print("-" * 40)
result = estimate_ew_from_depth(0.4, 0.12)
print(f"   EW estimate: {result['ew_mA']} mÅ")

result = curve_of_growth_regime(50, 5500)
print(f"   50 mÅ at 5500 Å: {result['regime']}")

print("\n" + "=" * 60)
print("All modular servers working!")
print("=" * 60)


### Configuring Multiple Servers

Now here's the key advantage of the modular approach: we can register **all these servers at once** with Claude Desktop. Let's update our setup function to handle multiple servers, then configure them all:

In [ ]:
def setup_multiple_servers(servers_dict, info=None):
    """
    Configure Claude Desktop with multiple MCP servers at once.
    
    Args:
        servers_dict: Dictionary mapping server names to file paths
                     e.g., {"doppler": "doppler_server.py", "broadening": "broadening_server.py"}
        info: System info (auto-detected if not provided)
    """
    if info is None:
        info = get_system_info()
    
    config_path = info['config_path']
    python_path = info['python_path']
    
    # Ensure config directory exists
    os.makedirs(os.path.dirname(config_path), exist_ok=True)
    
    # Load existing config
    if os.path.exists(config_path):
        with open(config_path, 'r') as f:
            try:
                config = json.load(f)
            except json.JSONDecodeError:
                config = {}
    else:
        config = {}
    
    if "mcpServers" not in config:
        config["mcpServers"] = {}
    
    # Add all servers
    for name, filepath in servers_dict.items():
        abs_path = os.path.abspath(filepath)
        config["mcpServers"][name] = {
            "command": python_path,
            "args": [abs_path]
        }
        print(f"  ✓ {name}: {filepath}")
    
    # Write config
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"\n✓ Configuration saved to:\n  {config_path}")
    return config_path


# Configure all our modular servers at once
print("Configuring multiple MCP servers:\n")

setup_multiple_servers({
    "doppler": "doppler_server.py",
    "broadening": "broadening_server.py",
    "analysis": "analysis_server.py"
})


In [ ]:
# Relaunch Claude Desktop with all servers
print("Relaunching Claude Desktop with multiple servers...")
print("(Quit Claude Desktop first if it's running)\n")
launch_claude_desktop()


After relaunching, Claude Desktop will have access to tools from **all three servers simultaneously**. You can mix and match:
- "What's the Doppler shift of H-alpha at 150 km/s, and how does that compare to thermal broadening at 6000K?"
- "If I measure an EW of 80 mÅ at 5500 Å, what curve of growth regime is that?"

Claude seamlessly uses tools from different servers as needed.

## Adding Data Resources

So far we've built tools—functions that perform calculations. MCP also lets you expose **resources**—data that Claude can read. This is particularly useful for reference data like line lists, atomic parameters, or observational catalogs.

Let's create a dedicated data server. This keeps our reference data separate from calculations, making it easy to update the line list without touching the calculation code:

In [ ]:
%%writefile data_server.py
#!/usr/bin/env python3
"""
MCP Server for Astronomical Reference Data

Provides line lists and atomic data as resources, plus tools
to search the data.
"""

import json
import numpy as np
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Astronomical Data")

# Reference data
IRON_LINES = [
    {"wavelength": 5250.21, "species": "Fe I", "ep_eV": 0.12, "loggf": -4.94,
     "notes": "Very weak, good for metal-rich stars"},
    {"wavelength": 5269.54, "species": "Fe I", "ep_eV": 0.86, "loggf": -1.32,
     "notes": "Moderate strength, widely used"},
    {"wavelength": 5324.18, "species": "Fe I", "ep_eV": 3.21, "loggf": -0.10,
     "notes": "Temperature sensitive (high excitation)"},
    {"wavelength": 5371.49, "species": "Fe I", "ep_eV": 0.96, "loggf": -1.65,
     "notes": "Clean profile, reliable"},
    {"wavelength": 5397.13, "species": "Fe I", "ep_eV": 0.92, "loggf": -1.99,
     "notes": "Weak, good for abundance"},
    {"wavelength": 5429.70, "species": "Fe I", "ep_eV": 0.96, "loggf": -1.88,
     "notes": "Reliable oscillator strength"},
    {"wavelength": 6137.69, "species": "Fe I", "ep_eV": 2.59, "loggf": -1.40,
     "notes": "Temperature indicator"},
    {"wavelength": 6252.56, "species": "Fe I", "ep_eV": 2.40, "loggf": -1.69,
     "notes": "Moderate strength"},
    {"wavelength": 6393.60, "species": "Fe I", "ep_eV": 2.43, "loggf": -1.43,
     "notes": "Clean, reliable"},
]

REFERENCE_LINES = [
    {"wavelength": 4861.33, "species": "H I", "name": "H-beta"},
    {"wavelength": 5889.95, "species": "Na I", "name": "Na D2"},
    {"wavelength": 5895.92, "species": "Na I", "name": "Na D1"},
    {"wavelength": 6562.80, "species": "H I", "name": "H-alpha"},
]

ATOMIC_MASSES = {
    "H": 1.008, "C": 12.011, "N": 14.007, "O": 15.999,
    "Na": 22.990, "Mg": 24.305, "Si": 28.086, "Ca": 40.078,
    "Ti": 47.867, "Cr": 51.996, "Fe": 55.845, "Ni": 58.693,
}


# Resources - data Claude can read
@mcp.resource("data://lines/iron")
def get_iron_lines() -> str:
    """Iron line list with atomic data for abundance analysis."""
    return json.dumps(IRON_LINES, indent=2)

@mcp.resource("data://lines/reference")
def get_reference_lines() -> str:
    """Common reference lines for wavelength calibration."""
    return json.dumps(REFERENCE_LINES, indent=2)

@mcp.resource("data://atomic-masses")
def get_atomic_masses() -> str:
    """Atomic masses in amu for thermal broadening calculations."""
    return json.dumps(ATOMIC_MASSES, indent=2)


# Tool to search the data
@mcp.tool()
def find_iron_lines(
    min_wavelength: float,
    max_wavelength: float
) -> dict:
    """
    Find iron lines within a wavelength range.
    
    Args:
        min_wavelength: Lower bound in Angstroms
        max_wavelength: Upper bound in Angstroms
        
    Returns:
        List of iron lines in range with atomic data
    """
    if min_wavelength >= max_wavelength:
        return {"error": "min must be less than max"}
    
    matches = [l for l in IRON_LINES 
               if min_wavelength <= l["wavelength"] <= max_wavelength]
    
    return {
        "range": f"{min_wavelength}-{max_wavelength} Å",
        "count": len(matches),
        "lines": matches
    }


if __name__ == "__main__":
    mcp.run()


### Testing the Data Server

Let's verify the data server works correctly. We'll test both the tool (find_iron_lines) and check that our reference data is accessible:

In [ ]:
# Test the data server
from data_server import find_iron_lines, IRON_LINES, ATOMIC_MASSES

print("Testing Data Server:\n")

result = find_iron_lines(5200, 5500)
print(f"Iron lines in 5200-5500 Å: {result['count']} found")
for line in result['lines']:
    print(f"  {line['wavelength']} Å: {line['notes']}")

print(f"\nAtomic masses available: {len(ATOMIC_MASSES)} elements")
print(f"Iron lines in database: {len(IRON_LINES)}")


### Adding the Data Server

Now let's add the data server to our configuration. Since we already have the other servers configured, we just need to add this one—Claude Desktop will connect to all of them:

In [ ]:
# Add the data server to our existing configuration
setup_mcp_server("data", "data_server.py")


In [ ]:
# Relaunch to pick up the new server
launch_claude_desktop()


Now in Claude Desktop, you can ask questions that combine data and calculations:
- "What iron lines are available between 5300 and 5500 Angstroms? Which would be best for abundance analysis?"
- "I want to measure iron abundance in a 5200K star. Recommend some good lines and tell me what thermal broadening to expect."
- "What's the atomic mass of calcium? How much thermal broadening would a Ca line at 4227 Angstroms have at 5800K?"

Claude can read your line lists, check the atomic data, and combine that information with calculations to give comprehensive answers.

## Best Practices for MCP Servers

As you build your own astronomical MCP servers, keep these guidelines in mind to create tools that work well with Claude and are easy to maintain.

### Write Clear Docstrings

Your docstring is how Claude understands your tool. Include what the function calculates and why it's useful, physical context for the calculation, what each parameter means with units, what the return values represent, and any limitations or caveats. Claude uses this information to decide when your tool is appropriate for a given question. A good docstring helps Claude make good decisions—and remember, you don't need to write separate schemas anymore!

### Always Include Type Hints

MCP needs type hints to generate tool schemas automatically. Without them, your tools won't work properly. Always specify parameter types and return types. The type hints become the schema that Claude uses to understand your function's inputs and outputs.

### Put Units in Parameter Names

Astronomical data uses many unit systems. Be explicit about units in your parameter names to prevent confusion when Claude constructs function calls:

```python
# Good - units are clear from the name
def thermal_broadening(
    wavelength_angstroms: float,
    temperature_kelvin: float,
    atomic_mass_amu: float
) -> dict:
    ...

# Bad - units are ambiguous, easy to make mistakes
def thermal_broadening(wavelength, temperature, mass):
    ...
```

### Validate Inputs and Return Error Dictionaries

Astronomical quantities have physical constraints—temperature must be positive, wavelengths must be positive, line depths must be between 0 and 1. Always check inputs and return helpful error messages. Use error dictionaries rather than raising exceptions, because MCP handles dictionaries gracefully and Claude can explain the error to the user:

```python
# Good - validates input and returns informative error
if temperature_kelvin <= 0:
    return {"error": "Temperature must be positive"}

if line_depth < 0 or line_depth > 1:
    return {"error": "Line depth must be between 0 and 1"}

# Bad - raises exception that may not be handled gracefully
if temperature_kelvin <= 0:
    raise ValueError("Temperature must be positive")
```

Returning error dictionaries gives Claude a message it can explain to the user in natural language.

### Return Rich Context

Don't just return numbers. Include context that aids interpretation:

```python
# Good - provides context for interpretation
return {
    "ew_mA": 45.2,
    "ew_angstroms": 0.0452,
    "caveat": "Line may be saturated—consider Voigt fitting",
    "regime": "transition region of curve of growth"
}

# Less helpful - just the number
return {"ew_mA": 45.2}
```

### Keep Tools Focused

Each tool should do one thing well. Instead of one massive `analyze_spectrum` function with many parameters and modes, provide focused tools that can be combined. Claude can chain `estimate_ew_from_depth`, then `curve_of_growth_regime`, then `thermal_broadening` as needed. This modularity makes tools easier to test, easier to document, and more flexible in how they can be combined.

The modular server approach we used in this lecture—separate files for Doppler, broadening, analysis, and data—exemplifies this principle at the server level too.

### Test Before Deploying

Verify your functions produce correct results for known cases. H-alpha at 100 km/s recession should show a shift of about 2.19 Å. Iron at 5000K should have thermal broadening of about 2.9 km/s. Write tests using known astronomical values to catch errors before they reach users.

## Summary

### Key Concepts

In this lecture, you've learned:

- **Model Context Protocol (MCP)**: A standard for packaging tools that LLMs can discover and use, enabling your code to work with multiple interfaces while writing it only once

- **No More Manual Schemas**: Unlike Part 2 where you wrote JSON schemas for each function, MCP automatically generates schemas from your type hints and docstrings—just write good documentation!

- **Cost Models**: The crucial difference between API-based access (pay per token) and Claude Desktop (subscription)—MCP with Claude Desktop enables interactive exploration at no incremental cost

- **Modular Server Architecture**: How to build focused, single-purpose servers that Claude Desktop can use simultaneously—avoiding code repetition and making maintenance easier

- **Tools vs Resources**: The distinction between calculations (tools) and data (resources), and how combining both creates more capable assistants

- **Automatic Configuration**: Using Python to detect your system, configure Claude Desktop, and launch the application—avoiding manual JSON editing and forgotten steps

### What You Can Now Do

After working through this material, you can package **any astronomical calculation** as an MCP tool accessible through natural conversation:

- **Database Queries & Coordinates**: Build a server that queries Simbad, converts coordinates between systems, or calculates angular separations—ask Claude "What's the galactic coordinates of Vega?" and get instant answers

- **Observation Planning**: Create tools that calculate airmass, find observable targets, or check Moon separation—"Is M31 observable tonight from Mauna Kea?"

- **Ephemerides**: Wrap Skyfield calculations for planet positions, rise/set times, or conjunction finding—"When does Jupiter rise tomorrow?"

- **Curve Fitting**: Package your optimization routines—"Fit a linear trend to this proper motion data"

- **Transit Analysis**: Expose transit fitting functions—"What planet radius does this 1.2% transit depth imply for a Sun-like star?"

- **Photometry**: Create PSF fitting and aperture photometry tools—"Estimate the magnitude from these source counts"

- **Spectroscopy**: Build the equivalent width and line profile tools we created today—"What's the thermal broadening for iron at 5500K?"

The pattern is always the same: write a function with good docstrings and type hints, add `@mcp.tool()`, and it becomes accessible through Claude Desktop. You now have the skills to turn any astronomical calculation into a conversational tool.

### Series Conclusion

Congratulations—you've completed this three-part tutorial series on building LLM-powered research agents!

**In Part 1**, you learned API fundamentals: authentication, requests, parameters, building conversations, managing context, prompting strategies, structured outputs, and vision models for image analysis.

**In Part 2**, you mastered function tools and Retrieval Augmented Generation (RAG): creating function schemas, building astronomical calculation tools, implementing document chunking and embedding-based search, and combining function tools with RAG for powerful research assistants.

**In Part 3 (this tutorial)**, you've learned to package your tools as standalone MCP servers that work with Claude Desktop, enabling subscription-based interactive exploration without per-token costs.

The MCP servers you built today can wrap any astronomical technique: query Simbad, convert coordinates, fit a transit model, measure equivalent widths—all accessible through natural conversation. Your code is no longer trapped in notebooks. It's infrastructure that can power your research in whatever interface suits the moment.

**Resources:**
- MCP Documentation: https://modelcontextprotocol.io/
- Anthropic Claude Documentation: https://docs.anthropic.com
- Claude Desktop Download: https://claude.ai/download

The tools will keep evolving, but the fundamentals you've learned—decomposing problems, writing clean code, validating results, documenting your work—those skills will serve you throughout your research career. Go build something amazing!


